## Extract eyes open and eyes closed and stop timestamps to crop data for resting state functional connectivity analysis
##### Input: *_preprocessed-raw.fif
#####  Output: *_RestingTStamps.csv
######   * = Subject e.g. 018 or C14 etc

In [3]:
import csv
import os
import sys
import mne
from mne.io import read_raw_fif
!pip install utils
#import utils
#import something to save in pickle

In [4]:
annotation_keys = {'eyes closed': 1, 'Trigger#1': 1, 'EYES CLOSED': 1, # eyes closed
                      'eyes open': 2, 'eyes opened': 2, 'Trigger#2': 2, 'EYES OPEN': 2, 'eyes openned':2, # eyes open
                      'stop': 9, 'Stop': 9, 'STOP': 9, 'Sop': 9, # stop
                      }

In [5]:
# Settings
# Read in processed raw data
#data_path = '../../Data/Processed Data/'
#This data path is local to sbflpc002
data_path = '/run/user/1000/gvfs/smb-share:server=research-cifs.nyumc.org,share=research/jwanglab/jwanglabspace/George Kenefati/Chronic Low Back Pain Study/Data/Processed Data'
save_path = '../../../Data/Eyes Timestamps/'

if not os.path.exists(save_path):
    os.makedirs(save_path)

In [6]:
file_list = os.listdir(data_path)
# Only select files that end in "raw.fif"
raw_fif_files = [file for file in file_list if file.endswith('raw.fif')]


In [229]:
file = raw_fif_files[34]
file_path = os.path.join(data_path, file)
raw = read_raw_fif(file_path, preload = False)


events_from_annot, event_dict = mne.events_from_annotations(raw, event_id = annotation_keys)
# Extract timestamps from the events
timestamps_with_id = [(event[0] / raw.info['sfreq'], event[2]) for event in events_from_annot]

# Get name of the file we're looking at
#print(file_path)

# Output timestamps with associated event IDs
print("Timestamps and Associated Event IDs:")
for timestamp, event_id in timestamps_with_id:
    print(f"Timestamp: {timestamp:.3f} seconds, Event ID: {event_id}")

Opening raw data file /run/user/1000/gvfs/smb-share:server=research-cifs.nyumc.org,share=research/jwanglab/jwanglabspace/George Kenefati/Chronic Low Back Pain Study/Data/Processed Data/C5._preprocessed-raw.fif...
    Range : 0 ... 642879 =      0.000 ...  1607.197 secs
Ready.
Used Annotations descriptions: ['stop']
Timestamps and Associated Event IDs:
Timestamp: 1539.100 seconds, Event ID: 9


##### Next, collect the timepoints for the eyes closed, eyes closed stop, eyes open, and eyes open stop

In [16]:
### Script for a normal timestamp
saved_times = [None, None, None, None]  # Initialize an array to store timestamps

for idx, (timestamp, event_id) in enumerate(timestamps_with_id):
    # Find first instance of event ID 1, then the first instance of event ID 9 afterwards
    # Find first instance of event ID 2, the the first instance of event ID 9 afterwards
    if event_id == 1:
        if saved_times[0] is None:  # Check if the first index is empty
            saved_times[0] = timestamp
    elif event_id == 9 and idx > 0 and timestamps_with_id[idx - 1][1] == 1:
        saved_times[1] = timestamp
    elif event_id == 2:
        if saved_times[2] is None:  # Check if the third index is empty
            saved_times[2] = timestamp
    elif event_id == 9 and idx > 0 and timestamps_with_id[idx - 1][1] == 2:
        saved_times[3] = timestamp

print("Saved Times:", saved_times)

Saved Times: [975.3975, None, 975.4425, None]


In [17]:
# Adjustments to saved times
saved_times[0] = 39.995
saved_times[1] = 387.595
saved_times[2] = 523.753
saved_times[3] = 945.398
print("Saved Times:", saved_times)

Saved Times: [39.995, 387.595, 523.753, 945.398]


In [18]:
# Function to extract subject ID from the file path
def extract_subject_id(file_path):
    file_name = file_path.split('/')[-1]  # Extract the file name
    subject_id = file_name.split('_')[0]  # Get the part before '_preprocessed-raw.fif'
    return subject_id

# Extract subject ID from the given file paths
subject_id = extract_subject_id(file_path)

# Define the CSV file name
csv_file_name = f"{subject_id}_RestingTStamps.csv"

# Construct the full file path
full_file_path = os.path.join(save_path, csv_file_name)

# Write data to the CSV file
with open(full_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Seconds"])

    for timestamp in saved_times:
        writer.writerow([timestamp])
    

In [20]:
file = raw_fif_files[34]
file_path = os.path.join(data_path, file)
raw = read_raw_fif(file_path, preload = False)


events_from_annot, event_dict = mne.events_from_annotations(raw)
# Extract timestamps from the events
timestamps_with_id = [(event[0] / raw.info['sfreq'], event[2]) for event in events_from_annot]

timestamps_event_ids_annotations = []
for idx, event in enumerate(events_from_annot):
    timestamp = event[0] / raw.info['sfreq']  # Timestamp in seconds
    event_id = event[2]  # Event ID
    #use event_dict to get the annotation
    timestamps_event_ids_annotations.append((timestamp, event_id))

# Print the extracted data
for timestamp, event_id in timestamps_event_ids_annotations:
    print(f"Timestamp: {timestamp:.3f}s, Event ID: {event_id}")

Opening raw data file /run/user/1000/gvfs/smb-share:server=research-cifs.nyumc.org,share=research/jwanglab/jwanglabspace/George Kenefati/Chronic Low Back Pain Study/Data/Processed Data/C5._preprocessed-raw.fif...
    Range : 0 ... 642879 =      0.000 ...  1607.197 secs
Ready.
Used Annotations descriptions: ['100048', '100480', 'back pinprick', 'back plastic', 'hand pinprick', 'hand plastic', 'stop', 'test']
Timestamp: 0.005s, Event ID: 1
Timestamp: 0.007s, Event ID: 1
Timestamp: 0.007s, Event ID: 1
Timestamp: 0.015s, Event ID: 1
Timestamp: 0.095s, Event ID: 1
Timestamp: 0.098s, Event ID: 1
Timestamp: 0.142s, Event ID: 1
Timestamp: 0.285s, Event ID: 1
Timestamp: 0.320s, Event ID: 1
Timestamp: 0.352s, Event ID: 1
Timestamp: 0.415s, Event ID: 1
Timestamp: 0.542s, Event ID: 1
Timestamp: 0.585s, Event ID: 1
Timestamp: 0.645s, Event ID: 1
Timestamp: 0.677s, Event ID: 1
Timestamp: 0.757s, Event ID: 1
Timestamp: 0.765s, Event ID: 1
Timestamp: 0.800s, Event ID: 1
Timestamp: 0.805s, Event ID: 1


In [14]:
event_dict

{'1000001': 1,
 '1000010': 2,
 '1100001': 3,
 '1100010': 4,
 'Med Pain Back': 5,
 'Med Pain Hand': 6,
 'No Pain Back': 7,
 'No Pain Hand': 8,
 'Stop': 9,
 'Yes Pain Back': 10,
 'Yes Pain Hand': 11,
 'eyes closed': 12,
 'eyes opened': 13}

In [10]:
raw.plot()

Using qt as 2D backend.


Channels marked as bad:
none


In [11]:
file = raw_fif_files[15]
file_path = os.path.join(data_path, file)
raw = read_raw_fif(file_path, preload = False)


events_from_annot, event_dict = mne.events_from_annotations(raw, event_id = annotation_keys)
# Extract timestamps from the events
timestamps_with_id = [(event[0] / raw.info['sfreq'], event[2]) for event in events_from_annot]

# Output timestamps with associated event IDs
print("Timestamps and Associated Event IDs:")
for timestamp, event_id in timestamps_with_id:
    print(f"Timestamp: {timestamp:.3f} seconds, Event ID: {event_id}")

Opening raw data file /run/user/1000/gvfs/smb-share:server=research-cifs.nyumc.org,share=research/jwanglab/jwanglabspace/George Kenefati/Chronic Low Back Pain Study/Data/Processed Data/C21_preprocessed-raw.fif...
    Range : 0 ... 627199 =      0.000 ...  1567.997 secs
Ready.


ValueError: Could not find any of the events you specified.